# Streaming inteligente: Eligiendo la mejor plataforma para ver películas

## Contexto y motivación

Nuestro proyecto da relevancia a un tema que notamos que suele ser un debate muy común hoy en día, e incluso nosotros mismos lo tenemos. El tema surgió cuando nos dimos cuenta de que todos compartíamos un gusto en común: ver películas. Sin embargo, nos percatamos de un problema frecuente existe una gran variedad de plataformas de streaming, pero no siempre sabemos cuál elegir, ya que cada una tiene algo especial que nos hace inclinarnos hacia ella, aunque no necesariamente sea la mejor opción, ya sea por preferencias personales o por motivos económicos.

Por eso mismo, quisimos investigar en base a qué se guía la gente hoy en día al momento de elegir una plataforma de streaming. Visitamos varias páginas de noticias y foros de discusión; entre ellas, destacamos una conversación en Reddit donde observamos que las personas tienden a elegir distintas plataformas según diversos aspectos "https://www.reddit.com/r/television/comments/1ju49ae/which_streaming_service_overall_is_the_best_in/?tl=es-419."
Como se ve en la discusión algunas se guían por la calidad de las películas o series, otras por la cantidad de contenido disponible y otras por los precios.

Asimismo, notamos que algunos sitios de recomendación se enfocan más en la cantidad de títulos y precios que en la calidad del contenido o otros factores que también son decisivos a la hora de elegir una plataforma.
"https://www.rastreator.com/telefonia/guias/plataformas-streaming"

Por ello, decidimos analizar estas opciones desde diferentes perspectivas: desde quienes priorizan la calidad sobre la cantidad de títulos, hasta quienes buscan una opción más familiar o económica.



## Datos

## **1. API de Watchmode**

Esta API tiene un registro sobre distintos títulos, ya sea serie o película, y la disponibilidad de estos en las distintas plataformas de streaming. Cuenta con distintos endpoints para poder buscar individualmente títulos, los detalles de estos, actores que participen en estos y demás, pero principalmente estaremos utilizando el endpoint de List Titles, el cual nos devolverá una lista de títulos que pueden ser filtrados según las características que se pidan, ya sea el tipo de este (si es una serie o película), la disponibilidad en cierta región o plataforma, por géneros y fechas. Los datos que entrega vienen en tipo JSON con los siguientes datos:

- id: Dato de tipo int, sirve como identificador único de cada película de la API

- title: Dato de tipo string, es el título de la película

- year: Dato de tipo int, indica el año en el que se estrenó la película

- imdb_id: Dato de tipo string, es el identificador de la película en IMDb

- tmdb_id: Dato de tipo string, es el identificador de la película en TMDb

- type: Dato categórico, es el tipo (pelicula, serie, etc.) que tiene en la API

- tmdb_type: Dato categórico, es el tipo que tiene en TMDb


##IMAGEN

Por ejemplo, la imagen muestra la lista de películas de Netflix obtenidas por la API.

##  **2. IMDb Non-Commercial Datasets**

Consisten en archivos de tipo .tsv (Valores separados por tabulaciones), que contienen distintos datos de la página IMDb, a continuación el listado de los archivos y su contenido:

### **title.basics.tsv**
Contiene datos básicos sobre los títulos (películas, series, etc.) registrados en IMDb, contiene las siguientes columnas:

- tconst: Dato de tipo string que sirve como identificador único para los títulos.

- titleType: Categórico, indica el tipo de el título (película, serie, etc.).

- primaryTitle: String, indica el nombre más popular de la película.

- originalTitle: String, indica el nombre original de la película.

- isAdult: Bool, indica si el título es o no para adultos.

- startYear: Int, año en el que se estrenó, en series es el año en que empezó.

- endYear: Int, año en el que terminó la serie, es nulo para los demás tipos.

- runtimeMinutes: Int, duración del título en minutos.

- genres: Array de strings, contiene hasta 3 géneros asociados al título.


#### IMAGEN

Fuente: IMDb (Non-Commercial Datasets)

### **name.basics.tsv**
Contiene datos sobre personas relacionadas a los títulos, contiene las siguientes columnas:

- nconst: String, sirve como identificador único para las personas.

- primaryName:  String, nombre con el que frecuentemente recibe crédito la persona.

- birthYear: Int, año de nacimiento de la persona.

- deathYear: Int, año de fallecimiento de la persona, es nulo si no aplica.

- primaryProfession: Array de strings, hasta 3 profesiones por las que es más conocida la persona.

- knownForTitles: Array de strings, títulos por los que es conocida la persona.


### IMAGEN

Fuente: IMDb (Non-Commercial Datasets)

### **title.ratings.tv**
Contiene los rankings y cantidad de votos que ha recibido cada título, contiene las siguientes columnas:

- tconst: String, sirve como identificador único de los títulos.

- averageRating: Float, puntuación promediada de los votos.

- numVotes: Int, cantidad de votos que ha recibido el título.


### **title.crew.tsv**
Contiene los identificadores de los directores y escritores de cada título, contiene las siguientes columnas:

- tconst: String, sirve como identificador único de los títulos.

- directors: Array de strings, identificadores de los directores del título.

- writers: Array de strings, identificadores de los escritores del título

### IMAGEN

Fuente: IMDb (Non-Commercial Datasets)

### **title.principals.tsv**
Relaciona los títulos con las personas, mostrando que rol tiene cada persona en los títulos, contiene las siguientes columnas:


- tconst: String, sirve como identificador único de los títulos.

- ordering: Int, sirve para diferenciar las filas, enumerando las relaciones entre personas y una misma película

- nconst: String, sirve como identificador único de las personas

- category: String, la categoría de trabajo que hace la persona


### IMAGEN

Fuente: IMDb (Non-Commercial Datasets)

## **Archivo IMDb Integración y preparación inicial**

En el notebook **imbd_netflix.ipynb**, el proceso se centra en extraer y combinar datos provenientes de los IMDb Non-Commercial Datasets.
El objetivo es construir una base de datos unificada con información de películas que también aparecían en la API de Watchmode.

Fuentes utilizadas

Se cargaron varios archivos .tsv de IMDb:

- title.basics.tsv → datos generales (año, género, duración, tipo)

- title.ratings.tsv → puntuaciones promedio y número de votos

- title.principals.tsv → actores, directores, equipo

- name.basics.tsv y title.crew.tsv para relacionar personas con títulos

#### **Procesos aplicados**

#### 1. **Lectura de archivos:**

In [ ]:
imdb_basics = pd.read_csv('title.basics.tsv', sep='\t')
imdb_ratings = pd.read_csv('title.ratings.tsv', sep='\t')
imdb_principals = pd.read_csv('title.principals.tsv', sep='\t')

Se usó sep='\t' porque los archivos están separados por tabulaciones.

#### 2. **Filtrado de películas:**

Solo se conservaron los registros cuyo titleType es 'movie', descartando series, cortos y otros tipos:

In [ ]:
imdb_basics = imdb_basics[imdb_basics['titleType'] == 'movie']

#### 3. **Selección de columnas relevantes:**

Se mantuvieron campos clave como tconst, primaryTitle, startYear, runtimeMinutes, genres.

#### 4. **Conversión de tipos de datos:**

Se convirtieron columnas numéricas (startYear, runtimeMinutes) a int, manejando valores nulos con NaN.

Las listas de géneros (genres) se separaron en strings para su futura comparación.

#### 5. **Unión de datasets:**

Los conjuntos title.basics y title.ratings se unieron por el identificador tconst:

In [ ]:
imdb_full = imdb_basics.merge(imdb_ratings, on='tconst', how='left')

Esto permitió incluir la calificación promedio y número de votos de cada película.

#### 6. **Integración con Watchmode:**

Luego se emparejó con el dataset de películas de Netflix obtenido mediante la API, usando la clave imdb_id ↔ tconst.

## **Archivo Limpieza – Depuración de datos**

El notebook limpieza_netflix.ipynb contiene la fase de limpieza y validación, donde se eliminaron, transformaron y estandarizaron los valores de los datos unificados.

Principales tareas de limpieza

#### 1. **Eliminación de valores nulos:**

Se detectaron columnas con valores faltantes (isna()) y se eliminaron o reemplazaron según el caso:

In [ ]:
df = df.dropna(subset=['title', 'year'])

En otras columnas menos críticas (como genres o runtimeMinutes), se imputaron valores:

In [ ]:
df['runtimeMinutes'].fillna(df['runtimeMinutes'].median(), inplace=True)

#### 2. **Manejo de duplicados:**

Se eliminaron duplicados basados en el identificador imdb_id o title:

In [ ]:
df = df.drop_duplicates(subset='imdb_id', keep='first')

#### 3. **Normalización de tipos de datos:**

- Columnas numéricas (year, runtimeMinutes, averageRating, numVotes) convertidas a tipos numéricos (int o float).

- Textos (title, genres, category) convertidos a minúsculas y sin espacios extra:

In [ ]:
df['genres'] = df['genres'].str.lower().str.strip()

#### 4. **Filtrado por rango válido:**

Se eliminaron registros con valores fuera de rango, por ejemplo:

- Películas con años fuera de 1900–2025.

- Duraciones extremadamente cortas o largas (outliers):

In [ ]:
df = df[(df['runtimeMinutes'] > 30) & (df['runtimeMinutes'] < 300)]

#### 5. **Creación de variables derivadas (feature engineering):**

- Se calcularon puntuaciones ponderadas según el número de votos.

- Se crearon variables como decada o genero_principal para análisis comparativos.

- Estandarización de nombres de plataformas (Netflix, Hulu, etc.) en una sola columna.

#### 6. **Validación final y exportación:**

Después de verificar los tipos y nulos:

In [ ]:
df.info()
df.to_csv('catalogo_limpio.csv', index=False)

Así se obtuvo una base limpia y unificada lista para análisis comparativos y visualizaciones.

### **Nota sobre el uso de los datos:**
Los datos utilizados en este proyecto provienen de Watchmode API y de los IMDb Non-Commercial Datasets. El uso de los datos de IMDb se realiza bajo las condiciones de sus Noncommercial Datasets, disponibles en IMDb Non-Commercial Datasets, exclusivamente para fines académicos y de investigación no comercial.

Los datos fueron procesados en dos etapas: primero se realizó la integración de archivos IMDb, extrayendo información de títulos, calificaciones y géneros mediante archivos .tsv. Luego, se realizo la limpieza y normalización, eliminando duplicados, valores nulos y registros inválidos, además de convertir los tipos de datos y crear variables derivadas para facilitar el análisis comparativo entre plataformas.
El resultado fue un dataset consolidado y validado que permitió calcular métricas como cantidad de películas, variedad de géneros y calificaciones promedio por plataforma, base para responder a las preguntas de investigación planteadas.

### Preguntas objetivos


1. ¿Qué plataforma presenta mayor cantidad de películas?
2. ¿Cúal es la plataforma que tiene mayor variedad de géneros?
3. ¿Cúal es la pataforma que presenta mayor cantidad de péliculas actuales   y cúal la que concentra más peliculas antiguas en su catálogo?
4. ¿Cúal es la plataforma que concentra la mayor cantidad de películas con mejores calificaciones?
5. ¿Qué actores se repiten más en el catálogo de cada plataforma?
6. ¿Qué directores se repiten más en el catálogo de cada plataforma?
7. ¿Qué plataforma presenta mejor la relación calidad y costo mensual percibida de su catálogo? (Especificar mensualidad estándar)
8. Si una persona quiere contratar un plan para su familia(el cual tiene hijos pequeños) que plataforma le recomendarías?
